# Stock market analytics and constructing diversified stock portfolio using graph theory

This tutorial will show you how to use graph analytics for analyzing stock market and how to construct diversified stock portfolio using **Leiden** community detection algorithm on a **S&P 500 dataset**.


## Prerequisites <a name="prerequisites"></a>

For this tutorial, we need to install:

- [Docker](https://docs.docker.com/get-docker/) - to run Memgraph, since Memgraph is a native Linux application and cannot be installed on Windows and macOS
- [Memgraph Platform](https://memgraph.com/docs/memgraph/installation) - the complete streaming graph application platform; follow the instructions to install Memgraph Platform with Docker for your OS
- [GQLAlchemy](https://pypi.org/project/gqlalchemy/)

> ### Memgraph Platform installation using Docker
>
> After we install Docker, we can run the Memgraph Platform container by running:
>
>```
>docker run -it -p 7687:7687 -p 7444:7444 -p 3000:3000 memgraph/memgraph-platform
>```
>
>**Memgraph Platform** contains:
>
>- **MemgraphDB** - the database that holds your data
>- **Memgraph Lab** - visual user interface for running queries and visualizing graph data (running at >`localhost:3000`)
>- **mgconsole** - command-line interface for running queries
>- **MAGE** - graph algorithms and modules library

We will use the **GQLAlchemy**'s object graph mapper (OGM) to connect to Memgraph and quickly execute **Cypher** queries. GQLAlchemy also serves as a Python driver/client for Memgraph. We can install it using:

```
pip install gqlalchemy
```

> You may need to install [CMake](https://cmake.org/download/) before installing GQLAlchemy.


## Connect to Memgraph with GQLAlchemy

In [ ]:
from gqlalchemy import Memgraph
memgraph = Memgraph("127.0.0.1", 7687)


Let's see if there's anything in the database. To do that, we can count the number of nodes in the database with the following Cypher query:
```
MATCH (n) RETURN count(n) AS number_of_nodes;
```

With the GQLAlchemy, you can execute that query and fetch its results:

In [ ]:
results = memgraph.execute_and_fetch(
    """
    MATCH (n) RETURN count(n) AS number_of_nodes ;
    """
)
print(next(results))


## Load the dataset

The simplest way to load a dataset into Memgraph is by using **Memgraph Lab** which is currently running on `localhost:3000`. The first thing you're going to see on that address is the **Quick Connect**. You just have to click `Connect`!

<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/2bb531a3459d5b07953ecb124b50ff677ee337bd/construct_portfolio/img/memgraph-lab-quick-connect.png" width="800" />

After you click `Connect`, head over to the `Datasets` tab, and load **S&P 500 stock prices** dataset.

<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/2bb531a3459d5b07953ecb124b50ff677ee337bd/construct_portfolio/img/memgraph-lab-load-dataset.png" width="800" />

Now, let's again check the **number of nodes** and **relationships** in the database:

In [ ]:
results = memgraph.execute_and_fetch(
    """
    MATCH (n) RETURN count(n) AS number_of_nodes ;
    """
)
print(next(results))


 In **Memgraph Lab** you can click on the **Graph Schema** tab and generate graph schema to see how the relationships are connecting the nodes in the database.

<img alt="movies_graph_model" src="https://raw.githubusercontent.com/memgraph/jupyter-memgraph-tutorials/b483044ba9c88a4a43c7544d27af750bc84a36f8/construct_portfolio/img/graph-data-model.png" width="800" />

## Stock market analysis with GQLAlchemy

Let's first list all the stocks in the dataset:

In [ ]:
results = memgraph.execute_and_fetch(
    """
    MATCH (s:Stock)
    RETURN collect(s.ticker) as tickers;
    """
)

print(next(results)['tickers'])


We can find stocks that had best daily return for each day since 01-05-2022, Let's see how:

In [ ]:
results = memgraph.execute_and_fetch(
    """      MATCH (s:Stock)-[r:Traded_On]->(d:TradingDay)
      WITH max(r.close/r.open) as max_daily_return, d
      MATCH (s:Stock)-[r:Traded_On]->(d:TradingDay) 
      WHERE (r.close/r.open) = max_daily_return AND d.date > "2022-05-01"
      RETURN DISTINCT collect(s.ticker) as ticker;
      """
)

print(next(results)['ticker'])


If we want to find stocks that had above average trading volume, we can do that with:

In [ ]:
results = memgraph.execute_and_fetch(
    """ MATCH (s:Stock)-[r:Traded_On]->(d:TradingDay)
      WITH avg(r.volume) as avg_volume, d
      MATCH (s:Stock)-[r:Traded_On]->(d:TradingDay) 
      WHERE r.volume > avg_volume AND d.date > "2022-05-01"
      RETURN DISTINCT s.ticker as ticker;
      """
)

tickers = [x['ticker'] for x in list(results)]
print(tickers)


## Constructing diverisified stock portfolio

The basic concept of constructing diversified stock portfolio is spreading your money among a variety of different investments in an effort to improve your risk-adjusted returns. One way of doing this is by finding stocks that behave similar and group them together to form a community. After we find stock communities, from each community we can pick stocks that will be part of our portfolio. In this tutorial, for finding communities we will use ***Leiden Algorithm***.

### Community detection with Leiden
The Leiden algorithm is community detection algorithm. It is based upon the *network modularity* metric, i.e. how well-connected the nodes within communities are compared to a network with randomized edges. It optimizes for maximum modularity by recursively merging communities into a single node and measuring the modularity of the condensed graph's clustering. 

Before we start using Leiden algorithm for construct our stock portfolio, we first need to load custom query module to memgraph. If you are interested in implementation, custom query module can be found in `construct_portfolio.py`.

In [ ]:
module_name = 'construct_portfolio.py'
file_text = open('construct_portfolio.py', "r").read().replace("'", '"')
query = f"CALL mg.create_module_file('{module_name}','{file_text}') YIELD *;"
list(memgraph.execute_and_fetch(query))


After we loaded custom query module, we can start calling procedure. Procedure can be called with following parameters.

- stock_tickers - list of stock tickers which we are taking in consideration for constructing portfolio.
- values - list of values used for calculating correlations between stocks (for example stock daily returns)
- n_trading_days_back - number of days taking in consideration while calculating correlations
- n_best_performing - number of best performing stocks to pick from each community
- resolution_parameter - Higher resolutions lead to more smaller communities, while lower resolutions lead to fewer larger communities.
- correlation_measure - measure to use for calculating correlations between stocks
- number_of_iterations - number of iterations used in leiden algorithm

In [ ]:
communities_record = list(memgraph.execute_and_fetch("""
    MATCH (s:Stock)-[r:Traded_On]->(d:TradingDay)
    WHERE d.date < "2022-04-27"
    WITH collect(s.ticker) as stocks,collect(r.close/r.open) as daily_returns
    CALL construct_portfolio.get(stocks,daily_returns,5,5,0.7,"pearson")
    YIELD community_index, community
    RETURN community_index, community;

    """
                                                     ))

for community in communities_record:
    print("Community_index:", community['community_index'],
          "Community members:", community['community'])


## Conclusion

In this tutorial, we analyzed stock market and used Leiden algorithm ,implemented in custom query module `construct_portfolio.py`, for finding stock communities so we could construct risk diversified portfolio. This is not financial advice, do your own research before investing. 